# RUSLE

Adaptado del codigo de [Lucas Rivero Iribarne](https://www.lucasriveroiribarne.com).

In [69]:
!pip install --upgrade pip
!pip install earthengine-API
!pip install --upgrade oauth2client
!pip install gcloud
!pip install geemap
!pip install seaborn
!pip install geopandas

## Spatial dataset

### Setup

In [70]:
# load libraries and API

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ee
import geemap
import geopandas as gpd
import numpy as np
import math
import os


In [71]:
# gee setup
ee.Authenticate()
ee.Initialize()

### Cargar area de influencia

In [72]:
#### Configuración
date_from = '2022-11-14'
date_to = '2023-11-14'

In [92]:
## cargar area de interes como geopandas dataframe y convertir a ee
# chile = gpd.read_file('/Users/nico/Desktop/Projects/RUSLE/02_input/Regiones/Regional.shp')
# region_metropolitana = chile[chile['codregion'] == 13]

# aoi = geemap.geopandas_to_ee(region_metropolitana)
aoi = (ee.FeatureCollection("FAO/GAUL/2015/level0")
                  .filter(ee.Filter.eq('ADM0_NAME', 'Chile')))

In [93]:
# Visualization
Map = geemap.Map()
Map.addLayer(aoi, {}, "Area of Interest")
Map.centerObject(aoi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Coleccion de datos

### Precipitacion 

In [94]:
# Precipitacón CHIRPS - https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY
pcp = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filter(ee.Filter.bounds(aoi))

Map = geemap.Map()

precipitationVis = {
  'min': 1.0,
  'max': 20.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

Map.addLayer(pcp.first().clip(aoi), vis_params=precipitationVis, name='Precipitación CHIRPS')
Map.add_colorbar(precipitationVis, label="Precipitación (mm)", layer_name="Precipitación CHIRPS")
Map.centerObject(aoi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [95]:
# Precipitacón CHIRPS - https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY
pcp = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filter(ee.Filter.bounds(aoi))

Map = geemap.Map()

precipitationVis = {
  'min': 1.0,
  'max': 20.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

Map.addLayer(pcp.first().clip(aoi), vis_params=precipitationVis, name='Precipitación CHIRPS')
Map.add_colorbar(precipitationVis, label="Precipitación (mm)", layer_name="Precipitación CHIRPS")
Map.centerObject(aoi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Suelo

Carbono, arena, limo y arcilla son provistos por el proyecto OpenLandMap a escala global basado en predicciones de machine learning de la compilacion de perfiles y muestras de suelo.

In [41]:
# Dado que la erosion es un proceso superficial, solo se usaran los datos superficiales (b0)
c_org = ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').select('b0').clip(aoi)
clay = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(aoi)
sand = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(aoi)

# Limo no se incluye como dataset, pero puede calcularse de la diferencia de particulas de arcilla y arena
silt = ee.Image(100).subtract(clay).subtract(sand).rename('silt')

In [42]:
# Visualization
viz = {'min': 0, 'max': 100, 'palette': ['red', 'yellow', 'green']}

Map = geemap.Map()
Map.addLayer(c_org, viz, "Carbono organico")
Map.addLayer(clay, viz, "Arcilla")
Map.addLayer(silt, viz, "Limo")
Map.addLayer(sand, viz, "Arena")
Map.add_colorbar(viz, label="%")
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

### Modelo digital de elevacion

In [43]:
# Modelo digital de elevación (DEM)
SRTM_dem = ee.Image('USGS/SRTMGL1_003')

DEMVis = {
  'min': 0.0,
  'max': 4000,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

colors = DEMVis['palette']
vmin = DEMVis['min']
vmax = DEMVis['max']

Map = geemap.Map()
Map.addLayer(SRTM_dem.clip(aoi), vis_params = DEMVis, name = "SRTM")
Map.add_colorbar(DEMVis, label = "Elevacion", layer_name = "SRTM")
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

### Landsat 8

In [44]:
# Landsat 8
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(optical_bands, None, True)

l8 = l8.map(apply_scale_factors)

Map = geemap.Map()

vizParams = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

Map.addLayer(l8.median().clip(aoi), vis_params=vizParams)
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

### Landcover

In [45]:
# MODIS
modis = ee.ImageCollection("MODIS/006/MCD12Q1").select('LC_Type1')

Map = geemap.Map()

igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

Map.addLayer(modis.filter(ee.Filter.bounds(aoi)), vis_params=igbpLandCoverVis, name='MODIS Land Cover')
Map.addLayer(aoi, {}, name='Region Metropolitana')
Map.add_legend(builtin_legend='MODIS/051/MCD12Q1')
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

## Factores RUSLE
### Erodabilidad del suelo (K)

La siguiente implementacion se realizo basada en [este tutorial](https://youtu.be/S6RR-pW6hSE?si=d0ZuYPAnsfWUrffH) que utiliza como referencia el metodo propuesto por [Williams (1995)](https://swat.tamu.edu/media/99192/swat2009-theory.pdf). Williams propone que:

$$
K_{USLE} = f_{csand}\cdot f_{cl-si}\cdot f_{orgc}\cdot f_{hisand}
$$

Donde $f_{csand}$ es un factor que da factores de erodibilidad del suelo bajos para suelos con altos contenidos de arena gruesa y valores altos para suelos con poca arena, $f_{cl-si}$ es un factor que da factores de erodibilidad del suelo bajos para suelos con altas relaciones arcilla-limo, $f_{orgc}$ es un factor que reduce la erodibilidad del suelo para suelos con alto contenido de carbono orgánico, y $f_{hisand}$ es un factor que reduce la erodibilidad del suelo para suelos con contenidos de arena extremadamente altos. Los factores se calculan:

$$
\begin{equation}
\begin{aligned}
f_{csand} &= \left(0.2+0.3\cdot exp\left[-0.256 \cdot m_s \cdot \left(1- \dfrac{m_{silt}}{100}\right)\right]\right)
\end{aligned}
\end{equation}
\\
\begin{equation}
\begin{aligned}
f_{cl-si} &= \left( \dfrac{m_{silt}}{m_{c}+m_{silt}}\right)^{0.3}
\end{aligned}
\end{equation}
\\
\begin{equation}
\begin{aligned}
f_{orgc} &= \left(1- \dfrac{0.25 \cdot orgC}{orgC + exp[3.72-2.95 \cdot orgC]}\right)
\end{aligned}
\end{equation}
\\
\begin{equation}
\begin{aligned}
f_{hisand} &= \left(1- \dfrac{0.7 \cdot \left(1- \dfrac{m_s}{100}\right)}{\left(1- \dfrac{m_s}{100}\right)+exp\left[ -5.51+22.9 \cdot \left(1- \dfrac{m_s}{100}\right)\right]}\right)
\end{aligned}
\end{equation}
$$

donde $m_s$, es el contenido porcentual de arena (partículas de 0,05-2,00 mm de diámetro), $m_{silt}$ es el contenido porcentual de limo (partículas de 0,002-0,05 mm de diámetro), $m_c$ es el contenido porcentual de arcilla (partículas de < 0,002 mm de diámetro), y $orgC$ es el contenido porcentual de carbono orgánico de la capa (%).


In [46]:
# calculando la erosividad 
f_csand = ee.Image(0.2).add(ee.Image(0.3).multiply(ee.Image(-0.256).multiply(silt.divide(100).subtract(1)).exp())).rename('f_csand')
f_cl_si = (silt.divide(clay.add(silt)).pow(0.3).rename('f_cl_si'))
f_orgc = (ee.Image(1).subtract(ee.Image(0.25).multiply(c_org).divide(c_org.add(ee.Image(3.72).subtract(ee.Image(2.95).multiply(c_org)).exp()))).rename('f_orgc'))
f_hisand = (
    ee.Image(1).subtract(
        ee.Image(0.7).multiply(ee.Image(1).subtract(sand.divide(100))).divide(
            ee.Image(1).subtract(sand.divide(100)).add(
                ee.Image(-5.51).add(ee.Image(22.9).multiply(ee.Image(1).subtract(sand.divide(100)))).exp()
            )
        )
    ).rename('f_hisand')
)

k_factor = f_csand.multiply(f_cl_si).multiply(f_orgc).multiply(f_hisand).rename('k_factor')

In [48]:
# Visualization
viz = {'min': 0.3, 'max': 0.5, 'palette': ['green', 'yellow', 'red']}

Map = geemap.Map()
Map.addLayer(k_factor, viz, "Erodabilidad del suelo (K)")
Map.add_colorbar(viz, label="Erodabilidad del suelo (K)", layer_name="Erodabilidad del suelo")
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

### Erosividad producida por la lluvia (R)

La erosividad por parte de la lluvia es la estimación de la perdida de suelo a causa de la precipitación (Uddin et al. 2018).

$R = 0.0483 * P^{1.610}$

Donde P corresponde a la precipitación anual

In [49]:
pcp_filtered = pcp.select('precipitation').filterDate(date_from, date_to).sum().clip(aoi)
# debo calcular promedio anual de varios años, el codigo de esta forma solo calcula para entre las fechas date_from y date_to

In [50]:
# R factor
r_factor = ee.Image(pcp_filtered.pow(1.610).multiply(0.0483)).rename('R_factor')

In [52]:
# Visualization
viz = {'min': 0, 'max': 5000, 'palette': ['green', 'yellow', 'red']}

Map = geemap.Map()
Map.addLayer(r_factor, viz, "Erosividad de la lluvia (R)")
Map.add_colorbar(viz, label="Erosividad de la lluvia (R)", layer_name="Erosividad de la lluvia (R)")
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

The values of the R factors seems to be to hight in comparison with the other factors. Is this correct?

### Longitud de la pendiente (L)

La longitud de la pendiente representa la distancia desde un pixel dado hasta hasta el punto de erosión potencial.

$L = (\frac{λ}{22.13})^m$

Donde λ corresponde a la longitud de la pendiente y *m* toma un valor entre 0.2 y 0.5en función de la pendiente.

In [ ]:
elev = SRTM_dem.select('elevation')
slope_deg = ee.Terrain.slope(elev).clip(aoi)
slope_perc = slope_deg.divide(180).multiply(math.pi).tan().multiply(100) # slope from degrees to percentage

m = ee.Image(1).where(slope_perc.gt(-1).And(slope_perc.lte(1)), 0.2).where(slope_perc.gt(1).And(slope_perc.lte(3)), 0.3).where(slope_perc.gt(3).And(slope_perc.lte(4.5)), 0.4).where(slope_perc.gt(4.5).And(slope_perc.lte(100)), 0.5)

In [ ]:
l_factor = (slope_perc.multiply(0).add(30).divide(slope_perc.multiply(0).add(22.13))).pow(m).rename("L_factor") # se multiplica por 0 y se suma 30 para crear un raster que pueda operar con la pendiente y 30 por el tamaño de pixel

In [54]:
# Visualization
viz = {'min': 1, 'max': 1.2, 'palette': ['green', 'yellow', 'red']}

Map = geemap.Map()
Map.addLayer(l_factor, viz, "Longitud de la pendiente (L)")
Map.add_colorbar(viz, label="Longitud de la pendiente (L)", layer_name="Longitud de la pendiente (L)")
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

### Inclinación de la pendiente (S)

El factor de inclinación de la pendiente representa la velocidad a la que puede fluir el agua en una superficie determinada, interactuando con el ángulo del suelo y afectando a la magnitud de la erosión del suelo.

$S = \frac{(0.43 + 0.3 * S + 0.043 * S^2)}{6.613}$

Donde *S* corresponde a la pendiente en porcentaje.

In [ ]:

# VALUES ARE TOO BIG
s_factor = slope_perc.pow(2).multiply(0.043).add(slope_perc.multiply(0.30)).add(0.43).divide(6.613).rename('Factor_S')



In [55]:
# Visualization
viz = {'min': 0, 'max': 45, 'palette': ['green', 'yellow', 'red']}

Map = geemap.Map()
Map.addLayer(s_factor, viz, "Inclinacion de la pendiente (S)")
Map.add_colorbar(viz, label="Inclinacion de la pendiente (S)", layer_name="Inclinacion de la pendiente (S)")
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

### Cobertura vegetal (C)

De Jong (1994) desarrolló la siguiente relación entre los valores del factor C calibrados en campo con el índice de vegetación de diferencia normalizada (NDVI) basado en Landsat para producir una superficie continua del factor C.

$C = 0.431 - 0.805 * NDVI$

$NDVI = \frac{NIR - Red}{NIR + Red}$

Donde NIR corresponde a la reflectividad en el infrarojo cercano y Red corresponde a la reflectividad en el rojo.

In [ ]:
l8_filtered = l8.filterDate(date_from, date_to).median().clip(aoi)
ndvi = l8_filtered.normalizedDifference(['SR_B5','SR_B4']).rename("NDVI")

In [ ]:
# De Jong (1994)
c_factor = ndvi.multiply(0).add(0.431).subtract(ndvi.multiply(0.805)).rename('C')

In [57]:
# Visualization
viz = {'min': 0, 'max': 0.5, 'palette': ['green', 'yellow', 'red']}

Map = geemap.Map()
Map.addLayer(c_factor, viz, "Cobertura vegetal (C)")
Map.add_colorbar(viz, label="Cobertura vegetal (C)", layer_name="Cobertura vegetal (C)")
Map.centerObject(aoi)
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

### Practicas control erosión (P)

El factor de práctica de control de la erosión refleja el impacto de las prácticas de apoyo sobre la tasa media anual de erosión, y también describe la relación entre las pérdidas de suelo en un campo concreto en el que se determina la práctica de control de la erosión.

En el presente ejercicio se consideró el producto de cobertura de suelos de MODIS (version mas actualizada del uso de suelo de este dataset es 2020) y se asignó un valor para cada cobertura basado en Chuenchum et al., 2019.


In [ ]:
# MODIS
modis = ee.ImageCollection("MODIS/006/MCD12Q1").select('LC_Type1').sort('system:time_start', False).first()

# P Factor de acuerdo a Chuenchum et al., 2019

lulc = (modis
        .clip(aoi)
        .rename('lulc'))

# create P Facor map using an expression
p_factor = lulc.expression(
            "(b('lulc') == 1) ? 0.8" +         # Evergreen Needleleaf Forests: dominated by evergreen conifer trees (canopy >2m). Tree cover >60%.
              ": (b('lulc') == 2) ? 0.8 " +    # Evergreen Broadleaf Forests: dominated by evergreen broadleaf and palmate trees (canopy >2m). Tree cover >60%.
              ": (b('lulc') == 3) ? 0.8" +     # Deciduous Needleleaf Forests: dominated by deciduous needleleaf (larch) trees (canopy >2m). Tree cover >60%.
              ": (b('lulc') == 4) ? 0.8" +     # Deciduous Broadleaf Forests: dominated by deciduous broadleaf trees (canopy >2m). Tree cover >60%.
              ": (b('lulc') == 5) ? 0.8" +     # Mixed Forests: dominated by neither deciduous nor evergreen (40-60% of each) tree type (canopy >2m). Tree cover >60%.
              ": (b('lulc') == 6) ? 0.8" +     # Closed Shrublands: dominated by woody perennials (1-2m height) >60% cover.
              ": (b('lulc') == 7) ? 0.8" +     # Open Shrublands: dominated by woody perennials (1-2m height) 10-60% cover.
              ": (b('lulc') == 8) ? 0.8" +     # Woody Savannas: tree cover 30-60% (canopy >2m).
              ": (b('lulc') == 9) ? 0.8" +     # Savannas: tree cover 10-30% (canopy >2m).
              ": (b('lulc') == 10) ? 0.8" +    # Grasslands: dominated by herbaceous annuals (<2m).
              ": (b('lulc') == 11) ? 1.0" +    # Permanent Wetlands: permanently inundated lands with 30-60% water cover and >10% vegetated cover.
              ": (b('lulc') == 12) ? 0.5" +    # Croplands: at least 60% of area is cultivated cropland.
              ": (b('lulc') == 13) ? 1.0" +    # Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles.
              ": (b('lulc') == 14) ? 0.5" +    # Cropland/Natural Vegetation Mosaics: mosaics of small-scale cultivation 40-60% with natural tree, shrub, or herbaceous vegetation.
              ": (b('lulc') == 15) ? 1.0" +    # Permanent Snow and Ice: at least 60% of area is covered by snow and ice for at least 10 months of the year.
              ": (b('lulc') == 16) ? 1.0" +    # Barren: at least 60% of area is non-vegetated barren (sand, rock, soil) areas with less than 10% vegetation.
              ": (b('lulc') == 17) ? 1.0" +    # Water Bodies: at least 60% of area is covered by permanent water bodies.
            ": 9999").rename('P').clip(aoi)     # NA

In [58]:
# Create a default map
Map = geemap.Map()

# Define the visualization parameters.
vizParams = {
  'min': 0,
  'max': 1,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}

# Add legend
Map.add_colorbar(viz, label="Practicas control erosión (P)", layer_name="Practicas control erosión (P)")

# Center the map and display the image.
Map.centerObject(aoi, 8)
Map.addLayer(p_factor, vizParams, 'Practicas control erosión (P)')

# Display the map
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

## Modelo RUSLE

In [ ]:
soil_loss = ee.Image(r_factor.multiply(k_factor).multiply(l_factor).multiply(s_factor).multiply(c_factor).multiply(p_factor).multiply(0.09)).rename('Perdida de suelo') # Se multiplica por 0.09 para calcular la erosión en el pixel de 30x30 m

In [59]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = soil_loss

# Define the visualization parameters.
vizParams = {
  'bands': ['Perdida de suelo'],
  'min': 0,
  'max': 50,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}

# Add legend
Map.add_colorbar(vizParams, label="Perdida de suelo (ton/ha/año)", layer_name="Perdida de suelo")

# Center the map and display the image.
Map.centerObject(aoi, 8)
Map.addLayer(image, vizParams, 'Perdida de suelo')

# Display the map
Map

Map(center=[-33.604245616814566, -70.62684991135265], controls=(WidgetControl(options=['position', 'transparen…

In [62]:
# Exportar imagen localmente
geemap.ee_export_image(soil_loss, '../03_output/RUSLE_soil_loss.tif', scale=90, crs=None, region=aoi.geometry(), file_per_band=False)


Generating URL ...
Please wait ...
Data downloaded to /Users/nico/Desktop/Projects/RUSLE/03_output/RUSLE_soil_loss.tif


## Referencias

1. Uddin, K., Abdul Matin, M., & Maharjan, S. (2018). Assessment of land cover change and its impact on changes in soil erosion risk in Nepal. Sustainability, 10(12), 4715.

2. Renard, K. G. (1997). Predicting soil erosion by water: a guide to conservation planning with the Revised Universal Soil Loss Equation (RUSLE). United States Government Printing.

3. Funk, C., Peterson, P., Landsfeld, M., Pedreros, D., Verdin, J., Shukla, S., ... & Michaelsen, J. (2015). The climate hazards infrared precipitation with stations—a new environmental record for monitoring extremes. Scientific data, 2(1), 1-21.

4. Boisier, J. P., Alvarez-Garretón, C., Cepeda, J., Osses, A., Vásquez, N., & Rondanelli, R. (2018, April). CR2MET: A high-resolution precipitation and temperature dataset for hydroclimatic research in Chile. In EGU General Assembly Conference Abstracts (p. 19739).

5. Zambrano, F., Wardlow, B., Tadesse, T., Lillo-Saavedra, M., & Lagos, O. (2017). Evaluating satellite-derived long-term historical precipitation datasets for drought monitoring in Chile. Atmospheric Research, 186, 26-42.

6. De Jong, S. M. (1994). Derivation of vegetative variables from a Landsat TM image for modelling soil erosion. Earth Surface Processes and Landforms, 19(2), 165-178.

7. Chuenchum, P., Xu, M., & Tang, W. (2019). Estimation of soil erosion and sediment yield in the Lancang–Mekong river using the modified revised universal soil loss equation and GIS techniques. Water, 12(1), 135.